In [1]:
import sys

sys.path.append('../')

from dataset import Pipeline, B, V
from dataset.models.tf import TFModel, RPN
from dataset.opensets import MNIST
from detection_mnist import DetectionMnist

In [2]:
IMAGE_SHAPE = (128, 128)
MNIST_PER_IMAGE = (5, 15)
RESIZE_TO = (14, 14)
ANCHORS_BATCH_SIZE = 16

BATCH_SIZE = 16

In [3]:
mnist = MNIST(batch_class=DetectionMnist)

ExtractingExtracting  ExtractingExtractingC:\Users\kozhevin\AppData\Local\Temp\t10k-images-idx3-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-images-idx3-ubyte.gz  

C:\Users\kozhevin\AppData\Local\Temp\t10k-labels-idx1-ubyte.gzC:\Users\kozhevin\AppData\Local\Temp\train-labels-idx1-ubyte.gz



In [4]:
# feed_dict = {'images': B('images'),
#              'anchors': B('anchors'),
#              'anchors_cls': B('clsf'),
#              'anchors_reg': B('reg'),
#              'anchors_batch': B('anchor_batch'),
#              }    

feed_dict = {'images': B('images'),
             'labels': B('labels'),
             'bboxes': B('bboxes')
             }    

In [5]:
placeholder_config = {'images': {'shape': IMAGE_SHAPE + (1,),
                                 'dtype': 'float32'},
                      'anchors': dict(),
                      'anchor_clsf': dict(),
                      'anchor_reg': dict(),
                      'anchor_labels': dict(),
                      'anchor_batch': dict()}

In [6]:
rpn_config = {'inputs': placeholder_config,
                'input_block/inputs': 'images',
                'batch_norm': {'momentum': 0.1},
                'head': {'map_shape': []},
                'loss': None,
                'optimizer': 'Adam'}

In [7]:
train_pp = (Pipeline()
            .init_variable('loss_history', init_on_each_run=list)
            .init_model('static', RPN, 'RPN', config=rpn_config)
            .generate_images(image_shape=IMAGE_SHAPE, 
                             n_digits=MNIST_PER_IMAGE, 
                             resize_to=RESIZE_TO)
            .train_model('RPN', 
                         feed_dict=feed_dict, 
                         fetches=['loss'], 
                         save_to=[V('loss_history')], mode='a')
            )

In [8]:
train_pp = train_pp << mnist.train

In [9]:
for i in range(10):
    train_pp.next_batch(16)

In [10]:
test_pp = (Pipeline()
            .init_variable('reg', init_on_each_run=list)
            .init_variable('clsf', init_on_each_run=list)
            .import_model('RPN', train_pp)
            .generate_images(image_shape=IMAGE_SHAPE, 
                             n_digits=MNIST_PER_IMAGE, 
                             resize_to=RESIZE_TO)
            .predict_model('RPN', 
                         feed_dict=feed_dict, 
                         fetches=['RPN/rpn_reg', 'RPN/rpn_clsf'], 
                         save_to=[V('reg'), V('clsf')], mode='a')
            )

In [11]:
test_pp = test_pp << mnist.train

In [12]:
test_pp.next_batch(10)